In [1]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf

In [2]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS_alone.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset= save['train_dataset']
    validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    test_old = save['test_dataset_evan']
    train_old = save['train_dataset_evan']
    del save

train_time = train_dataset['time']
train_data = train_dataset['data']
train_label = train_dataset['label']
validate_time = validate_dataset['time']
validate_data = validate_dataset['data']
validate_label = validate_dataset['label']
test_time = test_dataset['time']
test_data = test_dataset['data']
test_label = test_dataset['label']
test_old_data = test_old['data']
test_old_label = test_old['label']
train_old_data = train_old['data']
train_old_label = train_old['label']

In [3]:
#dataset normalize
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
print(mean.shape, std.shape)
train_data_n = (train_data - mean)/std
validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std

(70,) (70,)


In [4]:
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label.reshape(1, -1))
b_validate_label = pre.transform(validate_label.reshape(1, -1))
b_test_label = pre.transform(test_label.reshape(1, -1))

In [5]:
c_train_label = 1 - b_train_label[0]
c_validate_label = 1 - b_validate_label[0]
c_test_old_label = 1 - b_test_label[0]

In [6]:
#need to balanced the dataset
from unbalanced_dataset.over_sampling import SMOTE
sm = SMOTE(ratio = 0.03, kind='regular')
train_data_n_resample, c_train_label_resample = sm.fit_transform(train_data_n, c_train_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 174485, 1.0: 2502})
Finding the 5 nearest neighbours...
done!
Creating synthetic samples...Generated 2732 new samples ...
done!


/home/htan/.conda/envs/py27/lib/python2.7/site-packages/unbalanced_dataset/over_sampling/smote.py:311: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  X_new = np.zeros((n_samples, X.shape[1]))
/home/htan/.conda/envs/py27/lib/python2.7/site-packages/unbalanced_dataset/over_sampling/smote.py:317: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  size=n_samples)
/home/htan/.conda/envs/py27/lib/python2.7/site-packages/unbalanced_dataset/over_sampling/smote.py:323: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  size=n_samples)


In [7]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [8]:
v_train_label = makeIndicatorVars(c_train_label.reshape(-1, 1))
v_validate_label = makeIndicatorVars(c_validate_label.reshape(-1, 1))

Simple LSTM Model

In [9]:
feature_size = 70
label_size = 2
num_nodes = 140
batch_size = 30

In [13]:
graph = tf.Graph()
with graph.as_default():
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([feature_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([feature_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([feature_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([feature_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([1, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, label_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([label_size]))
    
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state
    
    #Input data
    
    tf_train_inputs = list()
    tf_train_labels = list()
    for i in range(batch_size):
        tf_train_data = tf.placeholder(tf.float32, shape=[1, feature_size])
        tf_train_label = tf.placeholder(tf.float32, shape=[1,label_size])
        tf_train_inputs.append(tf_train_data[i:i+1,:])
        tf_train_labels.append(tf_train_label[i:i+1,:])
    #Output data
    outputs = list()
    output = saved_output
    state = saved_state
    for i in range(batch_size):
        output, state = lstm_cell(tf_train_inputs[i], output, state)
        outputs.append(output)
        
    
    #state saving across unrollings
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        #Classifier
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, tf_train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)
    
   

In [14]:
num_steps = 7001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        for i in range(batch_size):
            offset = (step*batch_size + i) % (train_label.shape[0])
            # Generate a minibatch.
            batch_data = train_data_n[offset:(offset + 1), :].astype('float32')
            #print(batch_data.shape)
            batch_labels = v_train_label[offset:(offset + 1), :].astype('float32')
            print(batch_data.shape, batch_data.dtype)
            print(batch_labels.shape, batch_labels.dtype)
            feed_dict = {tf_train_data : batch_data, tf_train_label : batch_labels}
            _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
            mean_loss += l
        if step % 100 == 0:
            mean_loss = mean_loss / 100
            print('Mean loss at step %d: %f learning rate = %f' % (step, mean_loss, lr))
        

Initialized
(1, 70) float32
(1, 2) float32


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [1,70]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[1,70], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder', defined at:
  File "/home/htan/.conda/envs/py27/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/htan/.conda/envs/py27/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/htan/.conda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/htan/.conda/envs/py27/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/htan/.conda/envs/py27/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/htan/.conda/envs/py27/lib/python2.7/si